**En este notebook, nos enfocamos en mostrar el proceso intermedio de ETL y pre-procesamiento de datos, antes de realizar las transformaciones en Data Fusion. El resultado, son los archivos que se cargaran en el Google Cloud storage para hacer el proceso ETL.**

# 1. Pre-Procesamiento de los datos ambientales (SISAIRE)

## 1.1 Unión de archivos seleccionados en un solo CSV

In [1]:
import pandas as pd
import glob
import os

# Ruta donde están los CSV
ruta = "./Datos Ambientales/"  # cambia según tu estructura

# Buscar todos los CSV
archivos = glob.glob(os.path.join(ruta, "SISAIRE-*.csv"))

# Lista para almacenar los DataFrames
dataframes = []

for archivo in archivos:
    # Identificar el contaminante a partir del nombre del archivo
    if "NO2" in archivo:
        contaminante = "NO2"
        df = pd.read_csv(archivo)
        df["Concentracion"] = df["NO2"]
        df.drop(columns=["NO2"], inplace=True)
    elif "SO2" in archivo:
        contaminante = "SO2"
        df = pd.read_csv(archivo)
        df["Concentracion"] = df["SO2"]
        df.drop(columns=["SO2"], inplace=True)
    elif "O3" in archivo:
        contaminante = "O3"
        df = pd.read_csv(archivo)
        df["Concentracion"] = df["O3"]
        df.drop(columns=["O3"], inplace=True)
    else:
        continue  # por si hay archivos extra

    # Añadir columna de contaminante
    df["Contaminante"] = contaminante

    # Convertir fechas
    df["Fecha inicial"] = pd.to_datetime(df["Fecha inicial"], errors='coerce')
    df["Fecha final"] = pd.to_datetime(df["Fecha final"], errors='coerce')

    # Reorganizar columnas
    df = df[["Estacion", "Fecha inicial", "Fecha final", "Contaminante", "Concentracion"]]

    dataframes.append(df)

# Concatenar todos los DataFrames
sisaire_unificado = pd.concat(dataframes, ignore_index=True)

# Guardar el archivo unificado
sisaire_unificado.to_csv("SISAIRE_Unificado_2022_2024.csv", index=False)


## 2.1 Cambio de formato Fecha

In [2]:
import pandas as pd

# Cargar el archivo unificado
sisaire = pd.read_csv("SISAIRE_Unificado_2022_2024.csv")

# Asegurar formato de fecha
sisaire["Fecha final"] = pd.to_datetime(sisaire["Fecha final"], errors="coerce")

# Crear columna Año
sisaire["Ano"] = sisaire["Fecha final"].dt.year

# Crear columna Semestre (1 para enero-junio, 2 para julio-diciembre)
sisaire["Semestre"] = sisaire["Fecha final"].dt.month.apply(lambda x: 1 if x <= 6 else 2)

# Eliminar columnas innecesarias
sisaire.drop(columns=["Fecha inicial", "Fecha final"], inplace=True, errors='ignore')

# Reordenar columnas
sisaire = sisaire[["Estacion", "Contaminante", "Concentracion", "Ano", "Semestre"]]

# Guardar resultado
sisaire.to_csv("SISAIRE_Unificado_Transformado.csv", index=False)

print("Archivo transformado y guardado como SISAIRE_Unificado_Transformado.csv")
print(sisaire.head())


Archivo transformado y guardado como SISAIRE_Unificado_Transformado.csv
  Estacion Contaminante  Concentracion   Ano  Semestre
0   "USME"          NO2         46.634  2024         2
1   "USME"          NO2         33.659  2024         2
2   "USME"          NO2         34.975  2024         2
3   "USME"          NO2         28.018  2024         2
4   "USME"          NO2         29.146  2024         2


# 2. Pre-Procesamiento de los datos de salud

In [3]:
import pandas as pd
import gdown

# Diccionario con los archivos y sus semestres
archivos = {
    "1ohHEEl2F29MniNtDpCUzmJyyzVcZ7bJa": "2022-1",
    "1AWohxz-8OR0AQD7P8ClhDuhXNgs0e9kH": "2022-2",
    "1Iwx7sI5aseCrAQkS4gwhozO2KKtiQcZX": "2023-1",
    "1eFSlfAulm4D6fONnqX8HWHnFONO37NSm": "2023-2",
    "1C9BmidrpSd4mK1iEqLInOTG02Dn2H6s1": "2024-1",
    "1dOCWOPkFDOHlQ5zaQRiElde901hANe9T": "2024-2"
}

dataframes = []

for file_id, semestre in archivos.items():
    url = f"https://drive.google.com/uc?id={file_id}"
    output = f"morbilidad_{semestre}.csv"

    # Descargar el archivo
    gdown.download(url, output, quiet=False)

    # Leer el CSV
    df = pd.read_csv(output, low_memory=False, sep=';', encoding="utf-8")
    df.head()

    # Añadir columna semestre
    df["Semestre"] = int(semestre.split("-")[1])

    # Seleccionar solo las columnas relevantes
    df = df[[
        "ANO", "Sexo_Gen", "Edad_Quiquenio",
        "tipo_usuario_afiliacion", "prestador_localidad_codigo",
        "prestador_localidad_nombre", "tipo_atencion_nombre",
        "dxPrincipal_agrupacion1_nombre", "dxPrincipal_agrupacion2_nombre",
        "administradora_consolidada", "sum_atenciones", "Semestre"
    ]]

    dataframes.append(df)

# Unir todos los semestres
morbilidad_unificada = pd.concat(dataframes, ignore_index=True)

# Estandarizar nombres
morbilidad_unificada.rename(columns={
    "ANO": "Ano",
    "Sexo_Gen": "Sexo",
    "tipo_usuario_afiliacion": "Afiliacion",
    "prestador_localidad_nombre": "Localidad",
    "dxPrincipal_agrupacion1_nombre": "Enfermedad1",
    "dxPrincipal_agrupacion2_nombre": "Enfermedad2",
    "sum_atenciones": "NumeroAfectados"
}, inplace=True)

# Eliminar duplicados si los hay
morbilidad_unificada.drop_duplicates(inplace=True)

# Guardar archivo final
morbilidad_unificada.to_csv("Morbilidad_Unificada_2022_2024.csv", index=False)

print("Archivo unificado guardado como Morbilidad_Unificada_2022_2024.csv")
print(morbilidad_unificada.head())


Downloading...
From (original): https://drive.google.com/uc?id=1ohHEEl2F29MniNtDpCUzmJyyzVcZ7bJa
From (redirected): https://drive.google.com/uc?id=1ohHEEl2F29MniNtDpCUzmJyyzVcZ7bJa&confirm=t&uuid=e32c540e-0cc9-4d61-8716-4cd2ef58c7c6
To: /Users/aneira3/anaconda_projects/600602b5-2d91-4a23-8af7-9a43ffbba7d4/morbilidad_2022-1.csv
100%|██████████| 166M/166M [00:10<00:00, 16.5MB/s] 
Downloading...
From (original): https://drive.google.com/uc?id=1AWohxz-8OR0AQD7P8ClhDuhXNgs0e9kH
From (redirected): https://drive.google.com/uc?id=1AWohxz-8OR0AQD7P8ClhDuhXNgs0e9kH&confirm=t&uuid=ec426d53-1eb5-418d-b593-7edcbc53c128
To: /Users/aneira3/anaconda_projects/600602b5-2d91-4a23-8af7-9a43ffbba7d4/morbilidad_2022-2.csv
100%|██████████| 154M/154M [00:08<00:00, 19.0MB/s] 
Downloading...
From (original): https://drive.google.com/uc?id=1Iwx7sI5aseCrAQkS4gwhozO2KKtiQcZX
From (redirected): https://drive.google.com/uc?id=1Iwx7sI5aseCrAQkS4gwhozO2KKtiQcZX&confirm=t&uuid=f2e3fd39-625f-4095-96e5-175e0103955f
To: /

Archivo unificado guardado como Morbilidad_Unificada_2022_2024.csv
    Ano    Sexo Edad_Quiquenio    Afiliacion  prestador_localidad_codigo  \
0  2022   Mujer          0 a 4  Contributivo                          17   
1  2022   Mujer          0 a 4  Contributivo                          17   
2  2022   Mujer        10 a 14  Contributivo                          17   
3  2022  Hombre        50 a 54  Contributivo                          17   
4  2022   Mujer          5 a 9  Contributivo                          17   

       Localidad tipo_atencion_nombre  \
0  La Candelaria            Consultas   
1  La Candelaria    Hospitalizaciones   
2  La Candelaria            Consultas   
3  La Candelaria            Consultas   
4  La Candelaria            Consultas   

                                         Enfermedad1  \
0  Malformaciones Y Deformidades Congenitas Del S...   
1                Traumatismos Del Hombro Y Del Brazo   
2  Trastornos Neuroticos, Trastornos Relacionados...   
3  Tr

# 3. Pre-Procesamiento de los datos poblacionales

In [4]:
import pandas as pd

# Ruta al archivo Excel
archivo = "./Datos Poblacionales/Poblacion_Bogota_2018_2024.xlsx"

# Leer la hoja específica, saltando las primeras 11 filas
df = pd.read_excel(
    archivo,
    sheet_name="Localidades_edad_quinquenal",
    skiprows=11
)

# Filtrar solo las columnas que nos interesan
columnas = ["NOM_LOC", "AREA", "AÑO", "Hombres_0-4", "Mujeres_0-4"]
df = df[columnas]

# Eliminar filas vacías o de totales
df = df.dropna(subset=["NOM_LOC", "AÑO"])
df = df[df["NOM_LOC"] != "Total"]  # elimina posibles totales globales

# Estandarizar nombres de columnas
df.rename(columns={
    "NOM_LOC": "Localidad",
    "AREA": "Area",
    "AÑO": "Año",
    "Hombres_0-4": "Hombres_0a4",
    "Mujeres_0-4": "Mujeres_0a4"
}, inplace=True)

# Convertir a tipo entero donde aplique
for col in ["Año", "Hombres_0a4", "Mujeres_0a4"]:
    df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")

# Guardar en CSV limpio
df.to_csv("Poblacion_Bogota_2018_2024_Limpio.csv", index=False)

print("Archivo procesado y guardado como 'Poblacion_Bogota_2018_2024_Limpio.csv'")
print(df.head(10))


Archivo procesado y guardado como 'Poblacion_Bogota_2018_2024_Limpio.csv'
       Localidad                               Area   Año  Hombres_0a4  \
0        Usaquén                 Cabecera Municipal  2018        14692   
1        Usaquén  Centros Poblados y Rural Disperso  2018           15   
2        Usaquén                              Total  2018        14707   
3      Chapinero                 Cabecera Municipal  2018         4060   
4      Chapinero  Centros Poblados y Rural Disperso  2018           40   
5      Chapinero                              Total  2018         4100   
6       Santa Fe                 Cabecera Municipal  2018         3243   
7       Santa Fe  Centros Poblados y Rural Disperso  2018           64   
8       Santa Fe                              Total  2018         3307   
9  San Cristóbal                 Cabecera Municipal  2018        13756   

   Mujeres_0a4  
0        14307  
1           19  
2        14326  
3         3800  
4           44  
5        